# 优化器类

In [1]:
from abc import abstractmethod, ABC
import numpy as np

## Foundation

### Tensor

In [2]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)
        self.grad = None
        self.gradient_fn = lambda: None
        self.parents = set()

    def backward(self):
        if self.gradient_fn:
            self.gradient_fn()

        for p in self.parents:
            p.backward()

    def size(self):
        return len(self.data)

    def __str__(self):
        return str(self.data)

### Base Layer

In [3]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def parameters(self):
        return []

    def __str__(self):
        return ''

### Base Loss Function

In [4]:
class Loss(ABC):

    @abstractmethod
    def __call__(self, p: Tensor, y: Tensor):
        pass

### Base Optimizer

In [5]:
class Optimizer(ABC):

    def __init__(self, parameters, lr):
        self.parameters = parameters
        self.lr = lr

    @abstractmethod
    def backward(self):
        pass

## Data

### Features and Label

In [6]:
feature = Tensor([28.1, 58.0])
label = Tensor([165])

## Model

### Linear Layer

In [7]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        p = Tensor(x.data @ self.weight.data.T + self.bias.data)

        def gradient_fn():
            self.weight.grad = p.grad * x.data
            self.bias.grad = np.sum(p.grad, axis=0)

        p.gradient_fn = gradient_fn
        p.parents = {self.weight, self.bias}
        return p

    def parameters(self):
        return [self.weight, self.bias]

    def __str__(self):
        return f'weight: {self.weight}\nbias: {self.bias}'

### MSE Loss Function

In [8]:
class MSELoss(Loss):

    def __call__(self, p: Tensor, y: Tensor):
        mse = Tensor(((y.data - p.data) ** 2).mean())

        def gradient_fn():
            p.grad = -2 * (y.data - p.data)

        mse.gradient_fn = gradient_fn
        mse.parents = {p}
        return mse

### SGD Optimizer

In [9]:
class SGDOptimizer(Optimizer):

    def backward(self):
        for p in self.parameters:
            p.data -= p.grad * self.lr

## Configuration

### Learning Rate

In [10]:
LEARNING_RATE = 0.00001

## Testing

### Predicting

In [11]:
layer = Linear(feature.size(), 1)

prediction = layer(feature)

print(f'prediction: {prediction}')

prediction: [43.05]


### Calculating Loss

In [12]:
loss = MSELoss()

error = loss(prediction, label)

print(f'error: {error}')

error: 14871.802500000002


### Calculating Gradient and Updating Layer Parameters

In [13]:
optimizer = SGDOptimizer(layer.parameters(), lr=LEARNING_RATE)

error.backward()
optimizer.backward()

print(layer)

weight: [[0.5685359 0.641462 ]]
bias: [0.002439]
